### Introduction

In [13]:
import os
from dotenv import load_dotenv

In [15]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

In [33]:
prompt = "I want to open a restaurant for Nepali food. Suggest a fancy name for the restaurant. Give me a name only."
response = llm.invoke(prompt)
print(response)

content='Yak Cha Bar' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 61, 'total_tokens': 66, 'completion_time': 0.023298345, 'prompt_time': 0.003681218, 'queue_time': 0.052217442, 'total_time': 0.026979563}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--0f94ae31-82b3-4c38-809f-fff30b2f5d99-0' usage_metadata={'input_tokens': 61, 'output_tokens': 5, 'total_tokens': 66}


In [12]:
from langchain.prompts import PromptTemplate

prompt_template_resturant = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)


prompt_template_resturant.format(nationality="Nepali")

'I want to open a restaurant for Nepali food. Suggest a fancy name for the restaurant. Give me just a name.'

In [31]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_resturant)
chain.run("Nepali")

'"Kathmandu Oasis"'

### Sequence Chain

In [27]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

prompt_template_name = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)


prompt_template_items = PromptTemplate(
    input_variables = ['resturant name'],
    template = "Make up some menu items for {resturant name}. Return it as a comma seperated list."
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [35]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, items_chain])

response = chain.run("Nepali")

print(response)

Here are some menu items for "Kathmandu Palace":

Chicken Tikka Masala, Vegetable Samosas, Lamb Korma, Saag Aloo, Palak Paneer, Momo Platter, Nepal Chicken Curry, Gorkhali Lamb Chops, Garlic Naan, Vegetable Biryani, Tandoori Chicken, Kachoris, Kathmandu Veggie Wrap.

If you want me to categorize them into different sections (appetizers, main courses, sides, etc.), I can do that as well. Let me know!


**To show the intermediate output**

In [41]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

prompt_template_name = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="resturant_name")


prompt_template_items = PromptTemplate(
    input_variables = ['resturant_name'],
    template = "Make up some menu items for {resturant_name}. Return it as a comma seperated list."
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [44]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains = [name_chain, items_chain],
                              input_variables = ['nationality'], 
                              output_variables = ['resturant_name', "menu_items"])

response = chain.invoke({"nationality": "Nepali"})

print(response)

{'nationality': 'Nepali', 'resturant_name': '"Rashtradhwaja"', 'menu_items': 'Here\'s a list of menu items for "Rashtradhwaja", a fictional Indian restaurant:\n\nRajasthani Laal Maans, Chicken Tikka Tandoori, Palak Paneer, Amritsari Kulche, Baingan Bharta, Hyderabadi Haleem, Malabar Fish Curry, Mattar Paneer, Dahi Vada, Aloo Gobhi, Mysore Masala Dosa, Chicken Biryani.\n\nNote: \'Rashtradhwaja\' is the Hindi word for \'National Flag\'. The menu items are inspired by the diverse and rich culinary heritage of India.'}


### Agents

In [3]:
from langchain.agents import AgentType, load_tools, initialize_agent
from langchain_groq import ChatGroq

In [9]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # mimic thought and action,
    verbose=True
)

agent.invoke("When was Elon musk born? What is his age right now in 2025?")



> Entering new AgentExecutor chain...
Thought: We are looking for two pieces of information, Elon Musk's birthdate and his current age in 2025. To answer the first part of the question, we need to find Elon Musk's birthdate. To answer the second part of the question, we need to know Elon Musk's birthdate.

Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  ( EE-lon; born June 28, 1971) is a businessman. He is known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software compan

{'input': 'When was Elon musk born? What is his age right now in 2025?',
 'output': 'Elon Musk was born on June 28, 1971, and his age in 2025 is 54 years old.'}

In [19]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # mimic thought and action,
    verbose=True
)

agent.invoke("What was the GPD of Nepal in 2024?")



> Entering new AgentExecutor chain...
Thought: I need to find the GDP of Nepal in 2024. However, I'm only aware up to 2023 in my knowledge cutoff. I can use the search function to find the most recent data.

Action: Search
Action Input: "GDP of Nepal 2024"
Observation: ["World Economics estimates Nepal's 2024 GDP at $243 billion in PPP terms (Purchasing Power Parity) and an initial estimate of $255 billion for 2025. This figure ...", "World Economics estimates Nepal's 2024 GDP at $243 billion in PPP terms (Purchasing Power Parity) and an initial estimate of $255 billion for 2025.", "Nepal's economic growth accelerated in the first half of FY25 (H1FY25). Real GDP grew by 4.9 percent in H1FY25, up from 4.3 percent in H1FY24, ...", 'GDP in Nepal is estimated to be $148 Billion international dollars at the end of 2024 in GDP PPP terms.', 'Real GDP growth. Annual percent change. 4 ; GDP, current prices. Billions of U.S. dollars. 46.08 ; GDP per capita, current prices. U.S. dollars per cap

KeyboardInterrupt: 

### Memory

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()


In [23]:
from langchain.prompts import PromptTemplate

prompt_template_resturant = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)


prompt_template_resturant.format(nationality="Nepali")

'I want to open a restaurant for Nepali food. Suggest a fancy name for the restaurant. Give me just a name.'

In [24]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_resturant)
chain.run("Nepali")

C:\Users\samjh\AppData\Local\Temp\ipykernel_20212\1000647672.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_resturant)


'"Kathmandu Palace"'

In [26]:

chain = LLMChain(llm=llm, prompt=prompt_template_resturant, memory=memory)
name = chain.invoke("Mexican")

print(name)

{'nationality': 'Mexican', 'history': '', 'text': '"El Jardín de Sol"'}


In [27]:
name = chain.invoke("Nepali")

print(name)

{'nationality': 'Nepali', 'history': 'Human: Mexican\nAI: "El Jardín de Sol"', 'text': '"Kathmandu Kaash"'}


In [29]:
print(chain.memory.buffer)

Human: Mexican
AI: "El Jardín de Sol"
Human: Nepali
AI: "Kathmandu Kaash"


**In order to restrict the length of history data that gets fed to the model, we can use ConversationChain to restrict the past history to say, 5 conversations**

In [30]:
from langchain.chains import ConversationChain



llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\samjh\AppData\Local\Temp\ipykernel_20212\86030485.py:10: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo = ConversationChain(llm=llm)


In [31]:
convo.invoke("Who won the FIFA world cup 2022?")

{'input': 'Who won the FIFA world cup 2022?',
 'history': '',
 'response': "The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. Argentina emerged as the champions by defeating France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. This marked Argentina's third World Cup title, with their first wins in 1978 and 1986 under the leadership of Diego Maradona and Lionel Messi, respectively."}

In [32]:
convo.invoke("When is the next world cup?")

{'input': 'When is the next world cup?',
 'history': "Human: Who won the FIFA world cup 2022?\nAI: The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. Argentina emerged as the champions by defeating France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. This marked Argentina's third World Cup title, with their first wins in 1978 and 1986 under the leadership of Diego Maradona and Lionel Messi, respectively.",
 'response': 'The next FIFA World Cup is scheduled to take place in the United Arab Emirates and Saudi Arabia from November 21 to December 15, 2026. This will be the first time the tournament is hosted by multiple countries.'}

In [33]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': "Human: Who won the FIFA world cup 2022?\nAI: The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. Argentina emerged as the champions by defeating France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. This marked Argentina's third World Cup title, with their first wins in 1978 and 1986 under the leadership of Diego Maradona and Lionel Messi, respectively.\nHuman: When is the next world cup?\nAI: The next FIFA World Cup is scheduled to take place in the United Arab Emirates and Saudi Arabia from November 21 to December 15, 2026. This will be the first time the tournament is hosted by multiple countries.",
 'response': "Lionel Messi was the captain of the Argentina national team that won the 2022 FIFA World Cup. He was an instrumental figure in the team's success, leading them through the tournament with exceptional skill and determination."}

In [35]:
print(convo.memory.buffer)

Human: Who won the FIFA world cup 2022?
AI: The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. Argentina emerged as the champions by defeating France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. This marked Argentina's third World Cup title, with their first wins in 1978 and 1986 under the leadership of Diego Maradona and Lionel Messi, respectively.
Human: When is the next world cup?
AI: The next FIFA World Cup is scheduled to take place in the United Arab Emirates and Saudi Arabia from November 21 to December 15, 2026. This will be the first time the tournament is hosted by multiple countries.
Human: Who was the captain of the winning team?
AI: Lionel Messi was the captain of the Argentina national team that won the 2022 FIFA World Cup. He was an instrumental figure in the team's success, leading them through the tournament with exceptional skill and determination.


In [36]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)


llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

convo = ConversationChain(llm=llm, memory=memory)


C:\Users\samjh\AppData\Local\Temp\ipykernel_20212\2174504281.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [37]:
convo.invoke("Who won the FIFA world cup 2022?")

{'input': 'Who won the FIFA world cup 2022?',
 'history': '',
 'response': "The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. After a thrilling tournament, Argentina emerged as the champion by defeating France 4-2 in a penalty shootout, after the match ended 3-3 after extra time in the final on December 18, 2022. This marked Argentina's third World Cup title, with their previous wins being in 1978 and 1986, both under the legendary coach Carlos Bilardo and led by the iconic player Diego Maradona in 1986."}

In [38]:
convo.invoke("When is the next world cup?")

{'input': 'When is the next world cup?',
 'history': "Human: Who won the FIFA world cup 2022?\nAI: The FIFA World Cup 2022 was held in Qatar from November 20 to December 18, 2022. After a thrilling tournament, Argentina emerged as the champion by defeating France 4-2 in a penalty shootout, after the match ended 3-3 after extra time in the final on December 18, 2022. This marked Argentina's third World Cup title, with their previous wins being in 1978 and 1986, both under the legendary coach Carlos Bilardo and led by the iconic player Diego Maradona in 1986.",
 'response': 'The next FIFA World Cup is scheduled to take place in 2026, in the United States, Canada, and Mexico. This will be the first time that three countries will be co-hosting the FIFA World Cup. The tournament is scheduled to run from May 25 to July 15, 2026. This decision to host the tournament in three countries was made by the FIFA Congress in 2018.'}

In [39]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: When is the next world cup?\nAI: The next FIFA World Cup is scheduled to take place in 2026, in the United States, Canada, and Mexico. This will be the first time that three countries will be co-hosting the FIFA World Cup. The tournament is scheduled to run from May 25 to July 15, 2026. This decision to host the tournament in three countries was made by the FIFA Congress in 2018.',
 'response': "I don't have the information on who the captain of the winning team of the 2026 FIFA World Cup will be, as the tournament hasn't taken place yet. However, I can tell you that the 2022 FIFA World Cup was won by Argentina, and their captain at that time was Lionel Messi. But since the 2026 FIFA World Cup is still in the future, I don't have the information on the winning team or its captain."}